In [1]:
import tensorflow as tf
import random
from PIL import Image
from utils import visualization_utils as vis_util
import matplotlib.pyplot as plt
import cv2
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline

PATH_TO_CKPT =  'trainedModels/ssd_inception_RoadDamageDetector.pb' 

PATH_TO_LABELS = 'trainedModels/crack_label_map.pbtxt'

NUM_CLASSES = 8

In [2]:
detection_graph = tf.Graph()

with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [3]:
from utils import label_map_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = []
for index, name in enumerate(['D00','D01','D10', 'D11', 'D20', 'D40', 'D43', 'D44']):
    categories.append({'id': index+1, 'name':name})

category_index = { idx+1:item for idx, item in enumerate(categories)}

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = []
for index, name in enumerate(['D00 - linear crack - wheel mark',
                              'D01 - linear crack - join part',
                              'D10 - linear crack, equal ent', 
                              'D11 - linear crack - join part', 
                              'D20 - alligator crack', 
                              'D40 - rutting bump', 
                              'D43 - while line blur', 
                              'D44 - cross walk']):
    categories.append({'id': index+1, 'name':name})

category_index = { idx+1:item for idx, item in enumerate(categories)}

In [ ]:

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:

        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # тензоры класса скоров
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        cap = cv2.VideoCapture('video (2).mp4')
        
        asd = 0
        while True:
            # Читаем картинку
            
            ret, image = cap.read()

            image_np_expanded = np.expand_dims(image, axis=0)

            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            vis_util.visualize_boxes_and_labels_on_image_array(
                image,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                min_score_thresh=0.3,
                use_normalized_coordinates=True,
                line_thickness=8)
            cv2.imwrite('./our_race/' + str(asd) + '.jpg',image)

                 
            asd +=1
            cv2.imshow('Video', image)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyAllWindows()